In [0]:
!pip install pulp

import pulp
import requests
import numpy as np
import matplotlib.pyplot as plt

In [0]:
link = 'https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=json&q=MI&refine.state=MI&timezone=America/New_York'
response=requests.get(link)
data = response.json()

In [0]:
#returns the distance in km between two cities in the dataset using the haversine formula
#https://en.wikipedia.org/wiki/Haversine_formula
def dist(i,j):
  latdiff=np.radians(data[i]['fields']['latitude']-data[j]['fields']['latitude'])
  londiff=np.radians(data[i]['fields']['longitude']-data[j]['fields']['longitude'])

  a = np.sin(latdiff/2)**2 + np.cos(np.radians(data[i]['fields']['latitude'])
                                     ) * np.cos(np.radians(data[j]['fields']['latitude'])) * np.sin(londiff/2)**2
  c = 2 * np.arctan2(np.sqrt(a),  np.sqrt(1-a))
  return 6371 * c

In [0]:
#example of how to get the distance (only 3rd line is required)
print(data[5]['fields']['city'])
print(data[800]['fields']['city'])
dist(5,800)

In [0]:
#save the total number of zipcodes
num_zipcodes=len(data)
print(num_zipcodes)

In [0]:
#example to see what is saved in each data entry
data[800]

In [0]:
#to generate a random list and create a matrix of distances
zip_list=np.random.randint(low=0, high=len(data), size=20)

d = np.array([[dist(i,j) for j in zip_list] for i in zip_list])

#np.set_printoptions(precision=2, linewidth=140)
#print(d)

In [0]:
n = len(zip_list)
ZipCodes = range(n)

TSP = pulp.LpProblem("Travelling Salesperson Problem", pulp.LpMinimize)

x = pulp.LpVariable.dicts("x",(ZipCodes,ZipCodes),0,1,pulp.LpInteger)
p = pulp.LpVariable.dicts("p",ZipCodes,0,n,pulp.LpInteger)

TSP += sum(d[i,j]*x[i][j] for i in ZipCodes for j in ZipCodes)

for i in ZipCodes:
  TSP += sum(x[i][j] for j in ZipCodes) == 1
  TSP += sum(x[j][i] for j in ZipCodes) == 1
  TSP += x[i][i] == 0

TSP += p[0] == 1
for i in range(1,n):
  TSP += p[i] >=2

for i in range(1,n):
  for j in range(1,n):
    TSP += p[i]-p[j]+1 <= n*(1-x[i][j])


TSP.solve()

print("Status:", pulp.LpStatus[TSP.status])

print("Total distance is", pulp.value(TSP.objective))

for v in TSP.variables():
    if v.varValue > 0.1:
      print(v.name, "=", v.varValue)

In [0]:
tour=[]
for i in range(1,n+1):
  for j in range(n):
    if -0.1 < i - p[j].varValue < 0.1:
      tour.append(j)
tour.append(tour[0])

path=np.array([data[zip_list[i]]['fields']['geopoint'] for i in tour])

points=np.array([data[i]['fields']['geopoint'] for i in range(len(data))])

plt.plot(path[:,1],path[:,0], 'C3', lw=3)
plt.scatter(points[:,1],points[:,0])
plt.show()